# About 
Multivariate analysis includes bivariate analysis

# Libraries

In [1]:
%run general_fv.ipynb

22/06/30 13:52:39 WARN Utils: Your hostname, cesar-GL62M-7RDX resolves to a loopback address: 127.0.1.1; using 192.168.1.155 instead (on interface wlp2s0)
22/06/30 13:52:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/30 13:52:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/30 13:52:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/30 13:52:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
User information is ready!


# User-Defined Functions

# Data

In [2]:
sqlq = "SELECT * FROM `sima_station_CE` where datetime > \'2021-03-01\'"
data = ps_sqlq(sqlq, True)
data

/home/cesar/anaconda3/envs/spark_env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


DataFrame[datetime: timestamp, no: double, no2: double, nox: double, o3: double, pm10: double, pm25: double, prs: double, rainf: double, rh: double, so2: double, sr: double, tout: double, wdr: double, wsr: double]

In [5]:
data.columns[1:]

['no',
 'no2',
 'nox',
 'o3',
 'pm10',
 'pm25',
 'prs',
 'rainf',
 'rh',
 'so2',
 'sr',
 'tout',
 'wdr',
 'wsr']

# Multivariate analysis

## Numerical vs numerical

In [12]:
from pyspark.mllib.stat import Statistics
import pandas as pd

# Input
col_names = data.columns[1:]
corr_data = data[col_names]

# Numerical data from input
features = corr_data.rdd.map(lambda row: row[0:])

# Pearson correlation
corr_mat=Statistics.corr(features, method="pearson")

# Transformation into Pandas dataframe for visualization
corr_df = pd.DataFrame(corr_mat)
# Columns and index are set as input column names
corr_df.index, corr_df.columns = col_names, col_names

corr_df

22/06/30 15:48:12 WARN PearsonCorrelation: Pearson correlation matrix contains NaN values.


,no,no2,nox,o3,pm10,pm25,prs,rainf,rh,so2,sr,tout,wdr,wsr
no,1.000000,0.615414,0.880573,-0.288249,0.317214,0.409236,0.089494,NaN,0.091495,0.294322,0.020856,-0.199826,0.185962,-0.285341
no2,0.615414,1.000000,0.912553,-0.233210,0.399669,0.481692,0.173961,NaN,-0.005365,0.433560,-0.044674,-0.247341,0.262009,-0.411551
nox,0.880573,0.912553,1.000000,-0.288606,0.401095,0.497405,0.151922,NaN,0.046799,0.407377,-0.017705,-0.250009,0.252059,-0.390987
o3,-0.288249,-0.233210,-0.288606,1.000000,0.116739,-0.031195,0.173135,NaN,-0.446098,0.138175,0.497484,0.473257,-0.204234,0.559687
pm10,0.317214,0.399669,0.401095,0.116739,1.000000,0.582096,0.250836,NaN,-0.162756,0.343204,0.235921,0.095064,0.081816,0.059536
pm25,0.409236,0.481692,0.497405,-0.031195,0.582096,1.000000,0.187450,NaN,0.081819,0.318724,0.090489,-0.054939,0.109695,-0.175301
prs,0.089494,0.173961,0.151922,0.173135,0.250836,0.187450,1.000000,NaN,0.361402,0.167309,0.094186,0.415311,0.161435,0.267169
rainf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
rh,0.091495,-0.005365,0.046799,-0.446098,-0.162756,0.081819,0.361402,NaN,1.000000,-0.179859,-0.290818,-0.135244,0.056504,-0.250403
so2,0.294322,0.433560,0.407377,0.138175,0.343204,0.318724,0.167309,NaN,-0.179859,1.000000,0.186105,-0.013022,0.041799,-0.005368


## Categorical vs categorical

### Pearson’s Chi-squared test
Conduct Pearson’s independence test for every feature against the label. For each feature, the (feature, label) pairs are converted into a contingency matrix for which the Chi-squared statistic is computed. All label and feature values must be categorical.
The null hypothesis is that the occurrence of the outcomes is statistically independent.

In [18]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest

dataset = [[0, Vectors.dense([0, 0, 1])],
           [0, Vectors.dense([1, 0, 1])],
           [1, Vectors.dense([2, 1, 1])],
           [1, Vectors.dense([3, 1, 1])]]

dataset = spark.createDataFrame(dataset, ["label", "features"])
chiSqResult = ChiSquareTest.test(dataset, 'features', 'label')
chiSqResult.select("degreesOfFreedom").collect()[0]

Row(degreesOfFreedom=[3, 1, 0])
chiSqResult = ChiSquareTest.test(dataset, 'features', 'label', True)
row = chiSqResult.orderBy("featureIndex").collect()
row[0].statistic

4.0